In [1]:
try:
  from langchain.vectorstores import Chroma
except ImportError:
  !pip install langchain-community
  from langchain.vectorstores import Chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
# Import required libraries
try:
  from langchain_ollama import OllamaEmbeddings
except ImportError:
  !pip install langchain-ollama
  from langchain_ollama import OllamaEmbeddings

In [3]:
# Initialize the embeddings model (use the same model as during creation)
embeddings_model = OllamaEmbeddings(model="all-minilm")

In [31]:
if('google.colab' in str(get_ipython() ) ):
    environment= 'google'
else:
    import os
    if (os.environ.get('PWD')=='/kaggle/working'):
        environment= 'kaggle'
    else:
        environment= 'local'
print(environment)

if (environment=='google'):
  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/gu%C3%ADas_texto.zip?raw=true -O guías_texto.zip
  ! unzip guías_texto.zip
  ! rm guías_texto.zip

  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/normas_texto.zip?raw=true -O normas_texto.zip
  ! unzip normas_texto.zip
  ! rm normas_texto.zip

import os
try:
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain
except ImportError:
  ! pip install langchain-community
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import SnowballStemmer
ss = SnowballStemmer("spanish")

def tildesDiacríticas(texto):
    texto = re.sub("ñ","n",texto)
    texto = re.sub("Ñ","N",texto)
    texto = re.sub("á","a",texto)
    texto = re.sub("é","e",texto)
    texto = re.sub("í","i",texto)
    texto = re.sub("ó","o",texto)
    texto = re.sub("ú","u",texto)
    texto = re.sub("Á","A",texto)
    texto = re.sub("É","E",texto)
    texto = re.sub("Í","I",texto)
    texto = re.sub("Ó","O",texto)
    return re.sub("Ú","U",texto)


# Example text preprocessing function
def preprocess_text(text):
    # tildes y diacríticas
    text = tildesDiacríticas(text)

    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization (in this case, just splitting by space, but you could use a tokenizer)
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('spanish'))
    words = [word for word in words if word not in stop_words]

    # # Stemming (demasiado violento)
    # words = [ss.stem(word) for word in words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a single string
    return ' '.join(words)

def text_loader_normas(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      doc_number = 'AR ' + '.'.join(filename.split('_')[0].split('-'))

      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()

      # Preprocess the text
      doc_content[0].page_content = preprocess_text(doc_content[0].page_content)

      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

normas_path = './normas_texto'  # Replace with your directory path
normas_lista = []
normas_lista = text_loader_normas(normas_path, normas_lista)

def text_loader_guías(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      # doc_number = 'AR Guía' + '.'.join(filename.split('_')[0].split('-'))
      doc_number = 'Guía AR ' + filename.split('_')[0].split('-')[0][2:]

      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()

      # Preprocess the text
      doc_content[0].page_content = preprocess_text(doc_content[0].page_content)

      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

# same but for guías_texto
guías_path = './guías_texto'  # Replace with your directory path
guías_lista = []
guías_lista = text_loader_guías(guías_path, guías_lista)

normativa_lista = normas_lista + guías_lista


documents = normativa_lista

from langchain.text_splitter import RecursiveCharacterTextSplitter


google
--2024-11-24 14:25:58--  https://github.com/bettachini/nlpTP/blob/main/arn/gu%C3%ADas_texto.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/bettachini/nlpTP/raw/refs/heads/main/arn/gu%C3%ADas_texto.zip [following]
--2024-11-24 14:25:58--  https://github.com/bettachini/nlpTP/raw/refs/heads/main/arn/gu%C3%ADas_texto.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bettachini/nlpTP/refs/heads/main/arn/gu%C3%ADas_texto.zip [following]
--2024-11-24 14:25:59--  https://raw.githubusercontent.com/bettachini/nlpTP/refs/heads/main/arn/gu%C3%ADas_texto.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
! pip install chromadb
from langchain.vectorstores import Chroma

In [70]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=600)
split_documents = text_splitter.split_documents(documents)


vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings_model
                                  )

retriever = vectorstore.as_retriever()

In [ ]:
#Installs Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Runs Ollama service in the background
!ollama serve &>/dev/null&

# Downloads the pre-trained model
! ollama pull all-minilm

# Ensayo queries

In [ ]:
!ollama pull llama3.1

In [51]:
# !pip install langchain-ollama
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.1"
)

In [71]:
# prompt: Modify previous cell to change format_docs for something that uses the vectorstore

from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


template = """
            Sos un experto en seguridad nuclear  dispuesto a responder preguntas sobre las normas y guias emitidas por la Autoridad Regulatoria Nuclear de Argentina.Tu tarea es responder la pregunta provista abajo.

            Tambien se proveen una serie de documentos que un retrieval engine considero mas similares a la pregunta.

            Provees respuestas concisas siguiendo la logica "pensemos paso a paso" y la siguente estructura, donde DOCUMENTOS
            es una lista de los nombres de los documentos reglamentarios a partir de los cuales se elabora la respuesta.
            por favor seguí el formato siguiente;

            Pregunta:
            {query}

            Respuesta:

            Documentos relacionados:
            {docs}

            """
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "docs": retriever | format_docs,
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


# Use RetrievalQA chain
#rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [58]:
query = "¿La Entidad Responsable puede delegar la ejecución de tareas necesarias para la seguridad radiológica?"

result = rag_chain.invoke(query)
result

'Pregunta:\n            ¿La Entidad Responsable puede delegar la ejecución de tareas necesarias para la seguridad radiológica?\n\nRespuesta:\n\nSí, La Entidad Responsable puede delegar total o parcialmente la ejecución de tareas inherentes a la seguridad radiológica, siempre y cuando mantenga la totalidad de la responsabilidad sobre ellas.\n\nDocumentos relacionados:\ndebe promover cultura seguridad organizacion 20\nentidad responsable puede delegar total parcialmente ejecucion tareas inherentes seguridad radiologica mantiene totalidad responsabilidad\n\nrespectivamente segun corresponda 96 \nentidad responsable responsable seguridad radiologica deben asegurar adopten toda medidas reducir minimo probabilidad produzcan exposiciones medicas intencionales\nincluir objetivos procesos tareas materia seguridad radiologica definir estructura organizativa prever recursos implementacion 52'

In [33]:
!pip install rouge-score bert-score
from bert_score import score

from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f95dc8923192471bfd42a72734b881a4f01c1bf24c87350b35cb3ba8496c9b65
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [35]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Abrir por titulo
sh = gc.open("GoldStandardGPT4")

# Seleccionar primera hoja
worksheet = sh.sheet1


In [38]:
n=55
data_available=[]
for n in range(n+1):
  document=worksheet.acell(f"A{n+2}")
  question = worksheet.acell(f'B{n+2}').value
  gs_answer = val = worksheet.acell(f'C{n+2}').value
  data_available.append({"question":question,"gs_answer":gs_answer})

In [39]:
data_available

[{'question': '¿La Entidad Responsable puede delegar la ejecución de tareas necesarias para la seguridad radiológica?',
  'gs_answer': 'Pregunta: ¿La Entidad Responsable puede delegar la ejecución de tareas necesarias para la seguridad radiológica? Respuesta: La Entidad Responsable podrá delegar total o parcialmente la ejecución de tareas necesarias para la seguridad radiológica y nuclear de una instalación Clase I, pero mantendrá en su totalidad la responsabilidad correspondiente.\nDocumentos Relacionados: NORMA AR 0.0.1. LICENCIAMIENTO DE INSTALACIONES CLASE I'},
 {'question': '¿Qué requisitos debe cumplir una persona que solicite una licencia individual?',
  'gs_answer': 'Pregunta: ¿Qué requisitos debe cumplir una persona que solicite una licencia individual? Respuesta: La persona que solicite una licencia individual debe satisfacer los siguientes requisitos: a) Poseer formación básica. b) Haber realizado la capacitación complementaria, cuando corresponda, a propuesta de la Entidad 

In [59]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def generate_using_new_template(exp_number,comment):
  important_info=[]
  important_info.append(["Pregunta","GoldStandard","Respuesta","BertSCORE_P","BertSCORE_R","BertSCORE_F1","ROUGE1","ROUGE2","ROUGEL"])

  for n,q in enumerate(data_available):
      #Bertscore
      current_answer=rag_chain.invoke(q["question"])
      gs_answer=q["gs_answer"]
      print(f"Pregunta {n}")
      P, R, F1 = score([current_answer], [gs_answer], lang="es")
      #rouge
      scores = scorer.score(current_answer, gs_answer)

      print(f"P={P} R={R} F1={F1}")

      important_info.append([q["question"],q["gs_answer"],current_answer,str(P.item()),str(R.item()),str(F1.item()),str(scores['rouge1']),str(scores['rouge2']),str(scores['rougeL'])])

  with open(f'/content/NLP/results/results_experiment_{exp_number}.txt', 'w') as f:
    for line in important_info:
        linestr=[str(i) for i in line]
        linea=";".join(linestr)
        f.write(f"{linea}\n")
  with open(f'/content/NLP/results/metadata_results_experiment_{exp_number}.txt', 'w') as f:
    f.write(comment)




In [ ]:
generate_using_new_template(exp_number=5,comment="Experimento con split 5000 y overlap 600")

Pregunta 0
P=tensor([0.6392]) R=tensor([0.5508]) F1=tensor([0.5917])
Pregunta 1
P=tensor([0.6184]) R=tensor([0.5374]) F1=tensor([0.5750])
Pregunta 2
P=tensor([0.7500]) R=tensor([0.7625]) F1=tensor([0.7562])
Pregunta 3
P=tensor([0.6724]) R=tensor([0.5795]) F1=tensor([0.6225])
Pregunta 4
P=tensor([0.6164]) R=tensor([0.5563]) F1=tensor([0.5848])
